In [31]:
import argparse, sys, os, errno
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] [%(levelname)s] %(name)s: %(message)s')
import yaml
import shutil
import shlex
import subprocess
import json
import subprocess
from shlex import quote
from copy import deepcopy

In [24]:
parser = argparse.ArgumentParser(description='exSeek main program')

parser.add_argument('step', type=str, 
    choices=('quality_control', 'quality_control_clean',
    'fastq_to_fasta', 'prepare_genome', 'bigwig',
    'mapping', 'count_matrix', 'call_domains', 'combine_domains',
    'normalization', 'feature_selection', 
    'differential_expression', 'evaluate_features', 'igv',
    'update_sequential_mapping', 'update_singularity_wrappers')
)
parser.add_argument('--dataset', '-d', type=str, required=True,
    help='dataset name')
parser.add_argument('--config-dir', '-c', type=str, default='config',
    help='directory for configuration files')
parser.add_argument('--cluster', action='store_true', help='submit to cluster')
parser.add_argument('--cluster-config', type=str, 
    help='cluster configuration file ({config_dir}/cluster.yaml by default)')
parser.add_argument('--cluster-command', type=str,
    help='command for submitting job to cluster (default read from {config_dir}/cluster_command.txt')
parser.add_argument('--singularity', action='store_true',
    help='use singularity')
args, extra_args = parser.parse_known_args(['feature_selection','-d','scirep','--config-dir','/home/xieyufeng/ex/config/'])

In [18]:
logger = logging.getLogger('exseek')
snakefile = None


In [12]:

logger.info('read default config file')
with open('/home/xieyufeng/ex/snakemake/default_config.yaml', 'r') as f:
    default_config = yaml.load(f)
default_config

[2019-02-28 09:23:17,224] [INFO] exseek: read default config file


{'rna_types': ['rRNA',
  'lncRNA',
  'miRNA',
  'mRNA',
  'piRNA',
  'snoRNA',
  'snRNA',
  'srpRNA',
  'tRNA',
  'tucpRNA',
  'Y_RNA'],
 'call_domain_pvalue': '05',
 'distribution': 'ZeroTruncatedNegativeBinomial',
 'bin_size': 20,
 'cov_threshold': 0.2,
 'scale_method': 'robust',
 'classifiers': 'random_forest',
 'n_selects': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50],
 'resample_method': 'bootstrap',
 'select_methods': ['robust'],
 'zero_fraction_filter': True,
 'zero_fraction_filter_params': {'threshold': 0.8},
 'log_transform': True,
 'log_transform_params': {'base': 2},
 'rpm_filter': False,
 'rpm_filter_params': {'threshold': 10},
 'fold_change_filter': True,
 'fold_change_filter_params': {'direction': 'any'},
 'fold_change_filter_direction': ['any', 'up', 'down'],
 'scaler': 'robust',
 'scaler_params': {'robust': {'with_centering': True}},
 'selector': 'robust',
 'selector_params': {'robust': {'cv': {'splitter': 'stratified_shuffle_split',
    'n_splits': 10,
    'test_siz

In [20]:
# snakemake command
snakemake_args = ['snakemake', '-k', '--rerun-incomplete']
extra_config = {}

In [25]:
args.config_dir

'/home/xieyufeng/ex/config/'

In [27]:
if not os.path.isdir(args.config_dir):
    raise ValueError('cannot find configuration directory: {}'.format(args.config_dir))
configfile = os.path.join(args.config_dir, '{}.yaml'.format(args.dataset))

In [29]:
if not os.path.isfile(configfile):
    raise ValueError('cannot find configuration file: {} '.format(configfile))
with open(configfile, 'r') as f:
    config = default_config
    user_config = yaml.load(f)
    config.update(user_config)

In [30]:
config

{'rna_types': ['rRNA',
  'lncRNA',
  'miRNA',
  'mRNA',
  'piRNA',
  'snoRNA',
  'snRNA',
  'srpRNA',
  'tRNA',
  'tucpRNA',
  'Y_RNA'],
 'call_domain_pvalue': '05',
 'distribution': 'ZeroTruncatedNegativeBinomial',
 'bin_size': 20,
 'cov_threshold': 0.2,
 'scale_method': 'robust',
 'classifiers': 'random_forest',
 'n_selects': [10],
 'resample_method': 'bootstrap',
 'select_methods': ['robust'],
 'zero_fraction_filter': True,
 'zero_fraction_filter_params': {'threshold': 0.8},
 'log_transform': True,
 'log_transform_params': {'base': 2},
 'rpm_filter': False,
 'rpm_filter_params': {'threshold': 10},
 'fold_change_filter': True,
 'fold_change_filter_params': {'direction': 'any'},
 'fold_change_filter_direction': ['any', 'up', 'down'],
 'scaler': 'robust',
 'scaler_params': {'robust': {'with_centering': True}},
 'selector': 'robust',
 'selector_params': {'robust': {'cv': {'splitter': 'stratified_shuffle_split',
    'n_splits': 10,
    'test_size': 0.2}},
  'rfe': {'step': 0.2}},
 'n_fea

In [35]:
fold_change_filter_direction

NameError: name 'fold_change_filter_direction' is not defined

In [34]:
fold_change_params = deepcopy(config.get('fold_change_filter_params', {}))
fold_change_params['direction'] = wildcards.fold_change_filter_direction

NameError: name 'wildcards' is not defined

In [89]:
command = [
    os.path.join('/home/xieyufeng/ex/test/machine_learning.py'), 'cross_validation',
    '--matrix', '/home/xieyufeng/ex/output/scirep/matrix_processing/filter.null.Norm_CPM.Batch_Combat_1.domains_combined.txt',
    '--sample-classes', '/home/xieyufeng/ex/data/scirep/sample_classes.no_stage.txt',
    '--output-dir', '/home/xieyufeng/ex/test/cross_validation/',
    '--transpose',
    '--positive-class', 'Colorectal Cancer',
    '--negative-class', 'Healthy Control',
    '--cv-params', json.dumps(config['cv_params'])
]
if config['fold_change_filter']:
    command += ['--fold-change-filter', '--fold-change-filter-params', json.dumps(fold_change_params)]
if config['zero_fraction_filter']:
    command += ['--zero-fraction-filter', '--zero-fraction-filter-params', json.dumps(config['zero_fraction_filter_params'])]
if config['log_transform']:
    command += ['--log-transform', '--log-transform-params', json.dumps(config['log_transform_params'])]
if config['scaler']:
    command += ['--scaler', config['scaler'], '--scaler-params', json.dumps(config['scaler_params'].get(config['scaler'], {}))]
if config['selector']:
    command += ['--selector', config['selector'], 
        '--selector-params', json.dumps(config['selector_params'].get(config['selector'], {})),
        '--n-features-to-select', 10]
if config['grid_search']:
    command += ['--grid-search', '--grid-search-params', json.dumps(config['grid_search_params'])]
if config['sample_weight']:
    command += ['--sample-weight', config['sample_weight']]

In [80]:
config['classifier_params'].get(json.dumps(config['classifier']))

In [90]:
command += ['--classifier', 'random_forest', 
            '--classifier-params', json.dumps(config['classifier_params'].get('random_forest', {}))]
command = list(map(str, command))

In [91]:
command

['/home/xieyufeng/ex/test/machine_learning.py',
 'cross_validation',
 '--matrix',
 '/home/xieyufeng/ex/output/scirep/matrix_processing/filter.null.Norm_CPM.Batch_Combat_1.domains_combined.txt',
 '--sample-classes',
 '/home/xieyufeng/ex/data/scirep/sample_classes.no_stage.txt',
 '--output-dir',
 '/home/xieyufeng/ex/test/cross_validation/',
 '--transpose',
 '--positive-class',
 'Colorectal Cancer',
 '--negative-class',
 'Healthy Control',
 '--cv-params',
 '{"splitter": "stratified_shuffle_split", "n_splits": 50, "test_size": 0.2}',
 '--fold-change-filter',
 '--fold-change-filter-params',
 '{"direction": "any"}',
 '--zero-fraction-filter',
 '--zero-fraction-filter-params',
 '{"threshold": 0.8}',
 '--log-transform',
 '--log-transform-params',
 '{"base": 2}',
 '--scaler',
 'robust',
 '--scaler-params',
 '{"with_centering": true}',
 '--selector',
 'robust',
 '--selector-params',
 '{"cv": {"splitter": "stratified_shuffle_split", "n_splits": 10, "test_size": 0.2}}',
 '--n-features-to-select',


In [92]:
print(' '.join(map(quote, command)))

/home/xieyufeng/ex/test/machine_learning.py cross_validation --matrix /home/xieyufeng/ex/output/scirep/matrix_processing/filter.null.Norm_CPM.Batch_Combat_1.domains_combined.txt --sample-classes /home/xieyufeng/ex/data/scirep/sample_classes.no_stage.txt --output-dir /home/xieyufeng/ex/test/cross_validation/ --transpose --positive-class 'Colorectal Cancer' --negative-class 'Healthy Control' --cv-params '{"splitter": "stratified_shuffle_split", "n_splits": 50, "test_size": 0.2}' --fold-change-filter --fold-change-filter-params '{"direction": "any"}' --zero-fraction-filter --zero-fraction-filter-params '{"threshold": 0.8}' --log-transform --log-transform-params '{"base": 2}' --scaler robust --scaler-params '{"with_centering": true}' --selector robust --selector-params '{"cv": {"splitter": "stratified_shuffle_split", "n_splits": 10, "test_size": 0.2}}' --n-features-to-select 10 --grid-search --grid-search-params '{"param_grid": {"logistic_regression": {"C": [1e-05, 0.0001, 0.001, 0.01, 0.1

In [94]:
def summarize_feature_selection(output_dir):
    metrics = []
    features = []
    pbar = tqdm(unit='directory')
    dirstack = []
    for count_method in os.listdir(os.path.join(output_dir, 'select_preprocess_method', 'combined_score')):
        preprocess_method = open(os.path.join(output_dir, 'select_preprocess_method', 'combined_score', count_method, 'selected_methods.txt')).readline().strip()
        dirstack.append(os.path.join(output_dir, 'cross_validation', preprocess_method + '.' + count_method))
        for compare_group in os.listdir(dirstack[-1]):
            dirstack.append(os.path.join(dirstack[-1], compare_group))
            for feature_selection_method in os.listdir(dirstack[-1]):
                classifier, n_select, select_method, fold_change_direction  = feature_selection_method.split('.')
                metadata = {
                    'compare_group': compare_group,
                    'classifier': classifier,
                    'n_features': n_select,
                    'fold_change_direction': fold_change_direction,
                    'preprocess_method': preprocess_method,
                    'count_method': count_method
                }
                try:
                    df = pd.read_table(os.path.join(dirstack[-1], feature_selection_method, 'metrics.test.txt'), sep='\t')
                except IOError:
                    continue
                df_metadata = pd.DataFrame(index=np.arange(df.shape[0]))
                for key, val in metadata.items():
                    df_metadata[key] = val
                metrics.append(pd.concat([df_metadata, df], axis=1))
                pbar.update(1)
            dirstack.pop()
        dirstack.pop()
    metrics = pd.concat(metrics, axis=0)
    return metrics

metrics = summarize_feature_selection('output/'+dataset)

#newdf = df[(df['fold_change_direction']==value_change)&(df['classifier']==classifier_use)]
#metrics=pd.read_table('output/scirep/summary/cross_validation/metrics.train.txt')
newdf=metrics
newdf.index = newdf.compare_group
newdf['preprocess_methods'] = np.array([newdf.preprocess_method[i]+'.'+newdf.count_method[i] for i in range(newdf.shape[0])])
compare_list_use = np.array(['Normal-CRC','Normal-CRC_S1',#'Normal-PAAD',
                             'Normal-PRAD','Normal-HCC'])
tmpdict = {}
for i in np.unique(newdf.index): 
    if np.isin(i,compare_list_use):
        print(i)
        tmpdict[i] = np.unique(newdf.loc[i].preprocess_methods)

NameError: name 'dataset' is not defined

In [95]:
cd ~/ex

/home/xieyufeng/ex


In [97]:
import gc, argparse, sys, os, errno
from IPython.core.display import HTML,Image
from functools import reduce
import h5py
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook, show
output_notebook()
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from sklearn.metrics import roc_curve,roc_auc_score,auc
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
from sklearn.neighbors import NearestNeighbors
from bokeh.palettes import Category20c
from ipywidgets import interact,interactive, FloatSlider,IntSlider, RadioButtons,Dropdown,Tab,Text

Populating the interactive namespace from numpy and matplotlib


/apps/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Loading BokehJS ...

In [134]:
def embed_pdf_figure(width='640', height='480', title='Image'):
    data = BytesIO()
    plt.savefig(data, format='pdf', metadata={'Title': title})
    data = data.getvalue()
    data = 'data:application/pdf;base64,'+ str(b64encode(data), encoding='utf-8')
    display(HTML('<object width="{}" height="{}" data="{}" download="{}.pdf"></object>'.format(
        width, height, data, title)))
    plt.close()
    

from matplotlib.backends.backend_pdf import PdfPages, PdfFile
from IPython.display import HTML, display, FileLink
from base64 import b64encode, b64decode
from io import StringIO, BytesIO
from contextlib import contextmanager

@contextmanager
def embed_pdf_pages(width=960, height=480, title='Image'):
    data = BytesIO()
    try:
        pdf = PdfPages(data, metadata={'Title': title})
        yield pdf
    finally:
        pdf.close()
        data = data.getvalue()
        data = 'data:application/pdf;base64,'+ str(b64encode(data), encoding='utf-8')
        display(HTML('<object width="{}" height="{}" data="{}"></object>'.format(width, height, data)))
        plt.close()

@contextmanager
def embed_pdf_data(width=640, height=480, title='Image'):
    try:
        data = BytesIO()
        yield data
    finally:
        data = data.getvalue()

        data = 'data:application/pdf;base64,'+ str(b64encode(data), encoding='utf-8')
        display(HTML('<object width="{}" height="{}" data="{}"></object>'.format(width, height, data)))
        plt.close()

fonttitle = {'family':'Arial',
                  'weight' : 'normal', 
                  'size' : 8}
fontlabel = {'family':'Arial',
                  'weight' : 'normal', 
                  'size' : 6.5}
fontticklabel = {'family':'Arial',
                  'weight' : 'normal', 
                  'size' : 6.5}
fontlegend = {'family':'Arial',
                  'weight' : 'normal', 
              #'linewidth':0.5,
                  'size' : 6.5}
fontcbarlabel = {'family':'Arial',
                 'weight' : 'normal', 
                 #'Rotation' : 270,
                 #'labelpad' : 25,
                 'size' : 6.5}
fontcbarticklabel = {'family':'Arial',#Helvetica
                 'weight' : 'normal', 
                 'size' : 5.5}
def std_plot(ax,xlabel,ylabel,title=None,
             legendtitle=None,bbox_to_anchor=None,
             labelspacing=1.2,borderpad=0,handletextpad=0,legendsort=True,markerscale=None,
             xlim=None,ylim=None,
             xbins=None,ybins=None,
             cbar=None,cbarlabel=None,
             moveyaxis=False,sns=False,left=True,rotation=None):
    #plt.yticks([0,0.2,0.4,0.6,0.8,1.0],['0','0.2','0.4','0.6','0.8','1.0'])
    pyplot.draw()
    #plt.figure(linewidth=30.5)
    if xlim is not None:  
        ax.set(xlim=xlim)
    if ylim is not None:
        ax.set(ylim=ylim)
    #pyplot.draw()
    if xbins is not None:
        locator = MaxNLocator(nbins=xbins)
        locator.set_axis(ax.xaxis)
        ax.set_xticks(locator())
    if ybins is not None:
        locator = MaxNLocator(nbins=ybins)
        locator.set_axis(ax.yaxis)
        ax.set_yticks(locator())
    pyplot.draw()
    ax.set_xticks(ax.get_xticks())
    ax.set_yticks(ax.get_yticks())
    ax.set_xlabel(xlabel,fontdict = fontlabel,labelpad=5.5)
    ax.set_ylabel(ylabel,fontdict = fontlabel,labelpad=5.5)
    if rotation is not None:
        ax.set_xticklabels(ax.get_xticklabels(),fontticklabel,rotation=rotation)
    else:
        ax.set_xticklabels(ax.get_xticklabels(),fontticklabel)
    ax.set_yticklabels(ax.get_yticklabels(),fontticklabel)

    if moveyaxis is True:
        #fontticklabel 
        ax.spines['left'].set_position(('data',0))
    ax.spines['left'].set_visible(left)
    ax.spines['right'].set_visible(not left)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_linewidth(0.5)
    ax.spines['bottom'].set_linewidth(0.5)
    ax.spines['left'].set_linewidth(0.5)
    ax.spines['bottom'].set_color('k')
    ax.spines['left'].set_color('k')
    ax.spines['right'].set_color('k')
    
    ax.tick_params(direction='out', pad=2)
    #ax.spines['bottom']._edgecolor="#000000"
    #ax.spines['left']._edgecolor="#000000"
    if title is not None:
        ax.set_title(title,fontdict = fonttitle)
    if legendtitle is not None:
        #if legendloc is None:
        #    legendloc="best"
        legend = ax.legend(title=legendtitle,prop=fontlegend,
                      bbox_to_anchor=bbox_to_anchor,
                      labelspacing=labelspacing,borderpad=borderpad,handletextpad=handletextpad,
                      edgecolor="#000000",fancybox=False,markerscale=markerscale)
        ax.legend_.get_frame()._linewidth=0.5
        legend.get_title().set_fontweight('normal')
        legend.get_title().set_fontsize(6.5)
        if legendsort is True:
            # h: handle l:label
            h,l = ax.get_legend_handles_labels()
            l,h = zip(*sorted(zip(l,h), key=lambda t: int(t[0]))) 
            legend = ax.legend(h,l,title=legendtitle,prop=fontlegend,
                      bbox_to_anchor=bbox_to_anchor,
                      labelspacing=labelspacing,borderpad=borderpad,handletextpad=handletextpad,
                      edgecolor="#000000",fancybox=False,markerscale=markerscale)
            ax.legend_.get_frame()._linewidth=0.5
            legend.get_title().set_fontweight('normal')
            legend.get_title().set_fontsize(6.5)
        if sns is True:
            h,l = ax.get_legend_handles_labels()
            #l,h = zip(*sorted(zip(l,h), key=lambda t: int(t[0]))) 
            legend = ax.legend(h[1:],l[1:],title=legendtitle,prop=fontlegend,
                      bbox_to_anchor=bbox_to_anchor,
                      labelspacing=labelspacing,borderpad=borderpad,handletextpad=handletextpad,
                      edgecolor="#000000",fancybox=False,markerscale=markerscale)
            ax.legend_.get_frame()._linewidth=0.5
            legend.get_title().set_fontweight('normal')
            legend.get_title().set_fontsize(6.5)

    if cbar is not None:
        #locator, formatter = cbar._get_ticker_locator_formatter()
        #ticks, ticklabels, offset_string = cbar._ticker(locator, formatter)
        #cbar.ax.spines['top'].set_visible(False)
        #cbar.ax.spines['right'].set_visible(False)
        #cbar.ax.spines['bottom'].set_visible(False)
        #cbar.ax.spines['left'].set_visible(False)
        cbar.ax.tick_params(direction='out', pad=3,width=0,length=0)
        cbar.set_label(cbarlabel,fontdict = fontcbarlabel,Rotation=270,labelpad=7.5)
        cbar.ax.set_yticks(cbar.ax.get_yticks())
        cbar.ax.set_yticklabels(cbar.ax.get_yticklabels(),fontcbarticklabel)
    return ax

def gradient_func(val):
    return '<span style="background: linear-gradient(90deg, #d65f5f {0}%, transparent 0%)">{0:.3f}</span>'.format(val)

def display_dataframe(df, filename=None, encoding='utf-8', format='csv', type='button',gradientfunc=False, **kwargs):
    #display(df)
    #if isinstance(df, pd.DataFrame):
    #    display(df.style.set_caption(filename))
    #else:
    if gradientfunc == False:
        display(df.style.set_caption(filename))    
    else:
        display(df.style.format(gradient_func).set_caption(filename)) 
    if filename is None:
        filename = "dataframe"
    if format == 'csv':
        data = df.to_csv(**kwargs)
        mime_type = 'text/csv'
        filename = filename + '.csv'
    elif format == 'tsv':
        data = df.to_csv(**kwargs)
        mime_type = 'text/plain'
        filename = filename + '.txt'
    else:
        raise ValueError('unknown file format: {}'.format(format))
    data = 'data:{mime_type};base64,'.format(mime_type=mime_type) + str(b64encode(bytes(data, encoding=encoding)), encoding=encoding)
    if type == 'hyperlink':
        display(HTML('<a href=" " download={filename} target="_blank">{filename}</a >'.format(
            mime_type=mime_type, filename=filename, data=data)))
    elif type == 'button':
        button_id = 'button_{}'.format(np.random.randint(1000000000))
        display(HTML(r'<input type="button" id="{0}" value="Download">'.format(button_id)))
        display(HTML('''<script>
    document.getElementById("{button_id}").addEventListener("click", function(event){{
        var filename = "{filename}";
        var data = "{data}";
        const element = document.createElement('a');
        element.setAttribute('href', data);
        element.setAttribute('download', filename);
        element.style.display = 'none';
        document.body.appendChild(element);
        element.click();
        document.body.removeChild(element);
    }});
</script>'''.format(button_id=button_id, filename=filename, data=data)))
        
def log_transfrom(data,small=0.01):
    return np.log2(data+small)

In [120]:
#mirna_and_domains, mirna_and_long_bg, transcript_mirna
def interactive_config_settings(dataset,sequencing_type,classifier,value_change,example_cancer,reads_preprocess,stage_info,saveformat):
    if sequencing_type == 'short':
        exp_mx_name = 'mirna_and_domains'#'domains_combined'
    elif sequencing_type == 'mirna_and_long_bg':
        exp_mx_name = 'mirna_and_long_bg'
    elif sequencing_type == 'transcript_mirna':
        exp_mx_name = 'transcript_mirna'
    elif sequencing_type =='long':
        exp_mx_name = 'featurecounts'
    elif sequencing_type =='domain_only':
        exp_mx_name = 'domains_long'
    elif sequencing_type =='transcript':
        exp_mx_name = 'transcript'
    elif sequencing_type =='transcript_small':
        exp_mx_name = 'transcript_small'
    elif sequencing_type =='transcript_long_bg':
        exp_mx_name = 'transcript_long_bg'
    return dataset,sequencing_type,classifier,value_change,exp_mx_name,example_cancer,reads_preprocess,stage_info,saveformat

widget =interactive(interactive_config_settings,
           dataset= ['scirep','exorbase','exosome_small','pico_3v3'],
           sequencing_type=['short','mirna_and_long_bg','transcript_mirna','long','domain_only','transcript','transcript_small','transcript_long_bg'],
           classifier = ['random_forest','logistic_regression','linear_svm','decision_tree','logistic_regression_l1'],
           value_change = ['any','up','down'],
        example_cancer=['Normal-CRC','Normal-PAAD','Normal-PRAD','Normal-HCC'],
                   reads_preprocess=[True,False],
                   stage_info = ['No Stage','With Stage'],
                   saveformat=['.pdf','.eps'])  # if start from preprocessing
display(widget)


interactive(children=(Dropdown(description='dataset', options=('scirep', 'exorbase', 'exosome_small', 'pico_3v3'), value='scirep'), Dropdown(description='sequencing_type', options=('short', 'mirna_and_long_bg', 'transcript_mirna', 'long', 'domain_only', 'transcript', 'transcript_small', 'transcript_long_bg'), value='short'), Dropdown(description='classifier', options=('random_forest', 'logistic_regression', 'linear_svm', 'decision_tree', 'logistic_regression_l1'), value='random_forest'), Dropdown(description='value_change', options=('any', 'up', 'down'), value='any'), Dropdown(description='example_cancer', options=('Normal-CRC', 'Normal-PAAD', 'Normal-PRAD', 'Normal-HCC'), value='Normal-CRC'), Dropdown(description='reads_preprocess', options=(True, False), value=True), Dropdown(description='stage_info', options=('No Stage', 'With Stage'), value='No Stage'), Dropdown(description='saveformat', options=('.pdf', '.eps'), value='.pdf'), Output()), _dom_classes=('widget-interact',))

In [121]:
dataset,sequencing_type,classifier_use,value_change,exp_mx_name,example_cancer,reads_preprocess,stage_info,saveformat = widget.result
dataset,sequencing_type,classifier_use,value_change,exp_mx_name,example_cancer,reads_preprocess,stage_info,saveformat

('scirep',
 'short',
 'random_forest',
 'any',
 'mirna_and_domains',
 'Normal-CRC',
 True,
 'No Stage',
 '.pdf')

In [101]:
file_counts = 'output/'+dataset+'/summary/read_counts.txt'
file_length_path = 'output/'+dataset+'/stats/mapped_read_length_by_sample/'
file_length_path_insert = 'output/'+dataset+'/stats/mapped_insert_size_by_sample/'
plot_save_path = 'output/'+dataset+'/plots/'

    
import datetime
now = datetime.datetime.now()
timenow = '{}.{}.{}.{}:{}'.format(now.year,now.month,now.day,now.hour,now.minute)
savepath = 'output/candidate/'+dataset+'/'+dataset+'.'+sequencing_type+'.'+classifier_use+'.'+value_change+'.'+timenow+'/'
save_path = 'output/candidate/'+dataset+'/'+dataset+'.'+sequencing_type+'.'+classifier_use+'.'+value_change+'.'+timenow+'/'

if not os.path.exists(savepath):
    os.mkdir(savepath)

In [102]:
def summarize_feature_selection(output_dir):
    metrics = []
    features = []
    pbar = tqdm(unit='directory')
    dirstack = []
    for count_method in os.listdir(os.path.join(output_dir, 'select_preprocess_method', 'combined_score')):
        preprocess_method = open(os.path.join(output_dir, 'select_preprocess_method', 'combined_score', count_method, 'selected_methods.txt')).readline().strip()
        dirstack.append(os.path.join(output_dir, 'cross_validation', preprocess_method + '.' + count_method))
        for compare_group in os.listdir(dirstack[-1]):
            dirstack.append(os.path.join(dirstack[-1], compare_group))
            for feature_selection_method in os.listdir(dirstack[-1]):
                classifier, n_select, select_method, fold_change_direction  = feature_selection_method.split('.')
                metadata = {
                    'compare_group': compare_group,
                    'classifier': classifier,
                    'n_features': n_select,
                    'fold_change_direction': fold_change_direction,
                    'preprocess_method': preprocess_method,
                    'count_method': count_method
                }
                try:
                    df = pd.read_table(os.path.join(dirstack[-1], feature_selection_method, 'metrics.test.txt'), sep='\t')
                except IOError:
                    continue
                df_metadata = pd.DataFrame(index=np.arange(df.shape[0]))
                for key, val in metadata.items():
                    df_metadata[key] = val
                metrics.append(pd.concat([df_metadata, df], axis=1))
                pbar.update(1)
            dirstack.pop()
        dirstack.pop()
    metrics = pd.concat(metrics, axis=0)
    return metrics

metrics = summarize_feature_selection('output/'+dataset)

#newdf = df[(df['fold_change_direction']==value_change)&(df['classifier']==classifier_use)]
#metrics=pd.read_table('output/scirep/summary/cross_validation/metrics.train.txt')
newdf=metrics
newdf.index = newdf.compare_group
newdf['preprocess_methods'] = np.array([newdf.preprocess_method[i]+'.'+newdf.count_method[i] for i in range(newdf.shape[0])])
compare_list_use = np.array(['Normal-CRC','Normal-CRC_S1',#'Normal-PAAD',
                             'Normal-PRAD','Normal-HCC'])
tmpdict = {}
for i in np.unique(newdf.index): 
    if np.isin(i,compare_list_use):
        print(i)
        tmpdict[i] = np.unique(newdf.loc[i].preprocess_methods)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


Normal-CRC
Normal-CRC_S1
Normal-PRAD


In [105]:
newdf

,compare_group,classifier,n_features,fold_change_direction,preprocess_method,count_method,split,accuracy,average_precision,f1_score,precision,recall,roc_auc,preprocess_methods
compare_group,,,,,,,,,,,,,,
Normal-PRAD,Normal-PRAD,logistic_regression,10,up,filter.null.Norm_CPM.Batch_Combat_1,transcript_mirna,0,0.777778,0.868750,0.777778,0.700000,0.875,0.8375,filter.null.Norm_CPM.Batch_Combat_1.transcript...
Normal-PRAD,Normal-PRAD,logistic_regression,10,up,filter.null.Norm_CPM.Batch_Combat_1,transcript_mirna,1,0.833333,0.892045,0.800000,0.857143,0.750,0.8500,filter.null.Norm_CPM.Batch_Combat_1.transcript...
Normal-PRAD,Normal-PRAD,logistic_regression,10,up,filter.null.Norm_CPM.Batch_Combat_1,transcript_mirna,2,0.722222,0.758333,0.615385,0.800000,0.500,0.7500,filter.null.Norm_CPM.Batch_Combat_1.transcript...
Normal-PRAD,Normal-PRAD,logistic_regression,10,up,filter.null.Norm_CPM.Batch_Combat_1,transcript_mirna,3,0.722222,0.924145,0.736842,0.636364,0.875,0.9125,filter.null.Norm_CPM.Batch_Combat_1.transcript...
Normal-PRAD,Normal-PRAD,logistic_regression,10,up,filter.null.Norm_CPM.Batch_Combat_1,transcript_mirna,4,0.944444,1.000000,0.933333,1.000000,0.875,1.0000,filter.null.Norm_CPM.Batch_Combat_1.transcript...
Normal-PRAD,Normal-PRAD,logistic_regression,10,up,filter.null.Norm_CPM.Batch_Combat_1,transcript_mirna,5,0.777778,0.853869,0.750000,0.750000,0.750,0.8625,filter.null.Norm_CPM.Batch_Combat_1.transcript...
Normal-PRAD,Normal-PRAD,logistic_regression,10,up,filter.null.Norm_CPM.Batch_Combat_1,transcript_mirna,6,0.777778,0.875219,0.750000,0.750000,0.750,0.8625,filter.null.Norm_CPM.Batch_Combat_1.transcript...
Normal-PRAD,Normal-PRAD,logistic_regression,10,up,filter.null.Norm_CPM.Batch_Combat_1,transcript_mirna,7,0.777778,0.914423,0.750000,0.750000,0.750,0.9000,filter.null.Norm_CPM.Batch_Combat_1.transcript...
Normal-PRAD,Normal-PRAD,logistic_regression,10,up,filter.null.Norm_CPM.Batch_Combat_1,transcript_mirna,8,0.888889,0.965909,0.875000,0.875000,0.875,0.9625,filter.null.Norm_CPM.Batch_Combat_1.transcript...


In [103]:
tmpdict

{'Normal-CRC': array(['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg',
        'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna',
        'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains'],
       dtype=object),
 'Normal-CRC_S1': array(['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg',
        'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna',
        'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains'],
       dtype=object),
 'Normal-PRAD': array(['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg',
        'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna',
        'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains'],
       dtype=object)}

In [104]:
if dataset=='scirep':
    if stage_info =='No Stage':
        class_info = 'data/'+dataset+'/sample_classes.no_stage.txt'
    else:
        class_info = 'data/'+dataset+'/sample_classes.txt'

In [106]:
cpm_table_origin_ = pd.read_table('output/'+dataset+'/matrix_processing/filter.'+exp_mx_name+'.txt')
cpm_table_origin =  cpm_table_origin_/cpm_table_origin_.sum(axis=0)*10e6
length_tmp = np.array([cpm_table_origin.index[i].split('|')[-1] for i in range(cpm_table_origin.index.shape[0])]).astype('int')-\
np.array([cpm_table_origin.index[i].split('|')[-2] for i in range(cpm_table_origin.index.shape[0])]).astype('int')
rpkm_table_origin = (cpm_table_origin_.T/length_tmp*1000).T


def calculate_fc(table,original_index,sample_class, class_compare,cpm_table_origin):
    samples_use = np.array(sample_class.index)[np.isin(sample_class.label,class_compare)]
    sample_class_use = sample_class.iloc[np.isin(sample_class.label,class_compare)]
    sample_class_1_use = sample_class_use.iloc[np.where(sample_class_use.label ==class_compare[0])]
    sample_class_2_use = sample_class_use.iloc[np.where(sample_class_use.label ==class_compare[1])]
    table_selected = table.loc[original_index]
    length = np.array([cpm_table_origin.index[i].split('|')[-1] for i in range(cpm_table_origin.index.shape[0])]).astype('int')-\
np.array([cpm_table_origin.index[i].split('|')[-2] for i in range(cpm_table_origin.index.shape[0])]).astype('int')
    #print (length)

    rpkmtable = (cpm_table_origin.T/length).T*1000
    #display(rpkmtable)
    #display(table_selected)
    rpkmtable_ = pd.concat((rpkmtable.loc[original_index,sample_class_1_use.index],
               rpkmtable.loc[original_index,sample_class_2_use.index]),axis=1)
    cpmtable_ = pd.concat((cpm_table_origin.loc[original_index,sample_class_1_use.index],
               cpm_table_origin.loc[original_index,sample_class_2_use.index]),axis=1)
    if dataset =='scirep':
        rpkmtable=cpmtable_
    elif dataset=='exorbase':
        rpkmtable=rpkmtable_
    return np.mean(log_transfrom(table_selected.loc[:,sample_class_1_use.index]),axis=1) -\
np.mean(log_transfrom(table_selected.loc[:,sample_class_2_use.index]),axis=1),rpkmtable_,cpmtable_
def compare_features(path,sample_class,class_compare,compare_group,savepath=savepath,save_table=True):
    pbar = tqdm(unit='directory')
    feature_lists = {}
    for feature_selection_method in os.listdir(path):
        classifier, n_features, select_method,value_changes  = feature_selection_method.split('.')
        if classifier==classifier_use:
            n_features = int(n_features)
            if n_features > 10:
                continue
            # feature importance
            if value_changes == value_change:
                print (feature_selection_method)
                feature_lists[n_features] = pd.read_table(os.path.join(path,
                    feature_selection_method, 'feature_importances.txt'), header=None, index_col=0).iloc[:, 0]
                feature_lists[n_features].index = feature_lists[n_features].index.astype('str')
                pbar.update(1)
    # feature union set
    feature_set = reduce(np.union1d, [a.index.values for a in feature_lists.values()])
    # build feature importance matrix
    feature_matrix = pd.DataFrame(np.zeros((len(feature_set), len(feature_lists))),
                                  index=feature_set, columns=list(feature_lists.keys()))
    #print (feature_matrix.shape)
    for n_features, feature_importance in feature_lists.items():
        feature_matrix.loc[feature_importance.index.values, n_features] = feature_importance.values
    feature_matrix.columns = feature_matrix.columns.astype('int')
    feature_matrix.index = feature_matrix.index.astype('str')
    
    feature_matrix = feature_matrix.loc[:, feature_matrix.columns.sort_values().values]
    original_index = feature_matrix.index
    feature_info = feature_matrix.index.to_series().str.split('|', expand=True)
    feature_info.columns = ['gene_id', 'gene_type', 'gene_name', 'feature_id', 'transcript_id', 'start', 'end']

    feature_matrix.index = feature_info.loc[:, 'gene_name'].values \
        + '|' + feature_info.loc[:, 'gene_type'].values \
        + '|' + feature_info.loc[:, 'gene_id'].values \
    
    # build feature indicator matrix
    feature_indicator_matrix = pd.DataFrame(np.zeros((len(feature_set), len(feature_lists))),
                                  index=feature_set, columns=list(feature_lists.keys()))
    for n_features, feature_importance in feature_lists.items():
        feature_indicator_matrix.loc[feature_importance.index.values, n_features] = 1
    feature_indicator_matrix.columns = feature_indicator_matrix.columns.astype('int')
    feature_indicator_matrix = feature_indicator_matrix.loc[:, feature_indicator_matrix.columns.sort_values().values]
    
    feature_indicator_matrix.index = feature_info.loc[:, 'gene_name'].values \
        + '|' + feature_info.loc[:, 'gene_type'].values \
        + '|' + feature_info.loc[:, 'gene_id'].values
    #fig, ax = plt.subplots(figsize=(6, 8))
    #sns.heatmap(feature_indicator_matrix,
    #            cmap=sns.light_palette('green', as_cmap=True), cbar=False, ax=ax, linewidth=1)
    #ax.set_xlabel('Number of features')
    #ax.set_ylabel('Features')
    #display(processed_matrix)
    foldchange,rpkmtable,cpmtable = calculate_fc(processed_matrix, original_index,sample_class,class_compare,cpm_table_origin)
    foldchange.index = feature_matrix.index
    feature_matrix = pd.concat((feature_matrix,foldchange),axis=1)
    #display(feature_matrix) 
    if feature_matrix.shape[1]==2:
        feature_matrix.columns = np.concatenate((np.array([10]).astype('str'),np.array(['fold change'])))
    else:
        feature_matrix.columns = np.concatenate((np.arange(1,11).astype('str'),np.array(['fold change'])))
    #display (feature_matrix)
    feature_matrix.index = feature_info.loc[:, 'gene_name'].values \
        + '|' + feature_info.loc[:, 'gene_type'].values \
    + '|' + feature_info.loc[:, 'feature_id'].values \
        + '|' + feature_info.loc[:, 'transcript_id'].values
    #display (feature_matrix)
    display(feature_matrix.style\
        .background_gradient(cmap=sns.light_palette('green', as_cmap=True))\
        .set_precision(2)\
        .set_caption(path))
    pbar.close()
    if save_table:
        feature_matrix.to_csv(savepath+compare_group+'_feature_table.txt',sep='\t')
    return feature_matrix,feature_indicator_matrix,original_index,foldchange,rpkmtable,cpmtable

In [107]:
def scale(axis,table):
    '''
    axis: 0(by sample)/1(by feature)/2(both 0 and 1)
    '''
    scaler = StandardScaler()
    arr = np.array(table)
    if axis==0:
        return scaler.fit_transform(arr)
    elif axis==1:
        return scaler.fit_transform(arr.T).T
    elif axis==2:
        return scaler.fit_transform(scaler.fit_transform(arr).T).T
def clustermap(processed_matrix,featurename,sample_class, class_compare,compare_group,savepath=savepath,savefig=True,plot=False):
    samples_use = np.array(sample_class.index)[np.isin(sample_class.label,class_compare)]
    #samples_use = np.loadtxt(output_feature_selection_path+'{}/{}'.format(preprocess_method,compare_group)+)
    #display(sample_class.loc[samples_use])
    sample_class_use = sample_class.loc[samples_use].iloc[np.isin(sample_class.loc[samples_use].label,class_compare)]
    cpm_table = processed_matrix
    cpm_table_use = cpm_table.loc[featurename,samples_use]
    #display(cpm_table_use)
    cpm_matrix= log_transfrom(np.array(cpm_table_use))
    cpm_table_use.iloc[:,:] = RobustScaler().fit_transform(cpm_matrix.T).T
    rgblabel = np.repeat('r',samples_use.shape[0])
    rgblabel[np.where(sample_class_use.label==class_compare[-1])] = 'b'
    cpm_table_use_ = cpm_table_use
    whole_index = cpm_table_use.index
    cpm_table_use.index = np.array([cpm_table_use.index[i].split('|')[2]+'|'+cpm_table_use.index[i].split('|')[1] for i in range(cpm_table_use.index.shape[0]) ])
    if plot:
        sns_plot = sns.clustermap(cpm_table_use,cmap='vlag',figsize=(70,20),col_colors=rgblabel)
        if savefig:
            sns_plot.savefig(savepath+compare_group+'_clustermap'+saveformat, bbox_inches='tight')
    cpm_table_use_.index = whole_index
    return cpm_table_use,sample_class_use,cpm_table_use_


In [109]:
tmpdict

{'Normal-CRC': array(['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg',
        'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna',
        'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains'],
       dtype=object),
 'Normal-CRC_S1': array(['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg',
        'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna',
        'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains'],
       dtype=object),
 'Normal-PRAD': array(['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg',
        'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna',
        'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains'],
       dtype=object)}

In [113]:
preprocess_method.shape[0]

3

In [114]:
[preprocess_method[i].split('.')[-1] for i in range(preprocess_method.shape[0])]

['mirna_and_long_bg', 'transcript_mirna', 'mirna_and_domains']

In [118]:
np.where(np.array([preprocess_method[i].split('.')[-1] for i in range(preprocess_method.shape[0])])==exp_mx_name)

(array([], dtype=int64),)

In [119]:
preprocess_method[np.where(np.array([preprocess_method[i].split('.')[-1] for i in range(preprocess_method.shape[0])])==exp_mx_name)][0]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [122]:
output_feature_selection_path = 'output/'+dataset+'/cross_validation/'
feature_matrix,feature_indicator_matrix,original_index,foldchange,rpkmtable,cpm_table_use,sample_class_use,cpmtable,cpm_table_use_ = {},{},{},{},{},{},{},{},{}
for compare_group, preprocess_method in tqdm(tmpdict.items()):
    #print (compare_group,preprocess_method)
    preprocess_method = preprocess_method[np.where(np.array([preprocess_method[i].split('.')[-1] for i in range(preprocess_method.shape[0])])==exp_mx_name)][0]
    #compare_group = compare_group[0]            
    class_info = 'data/'+dataset+'/sample_classes.txt'
    if dataset =='scirep':
        if compare_group=='Normal-CRC':
            class_compare = np.array(['Colorectal Cancer', 'Healthy Control'])
            class_info = 'data/'+dataset+'/sample_classes.no_stage.txt'
        elif compare_group=='Normal-PAAD':
            class_compare = np.array(['Prostate Cancer', 'Healthy Control']) 
            class_info = 'data/'+dataset+'/sample_classes.no_stage.txt'
        elif compare_group=='Normal-PRAD':
            class_compare = np.array(['Pancreatic Cancer', 'Healthy Control']) 
            class_info = 'data/'+dataset+'/sample_classes.no_stage.txt'
        elif compare_group=='Normal-CRC_S1':
            class_compare = np.array([ 'Colorectal Cancer Stage 1', 'Healthy Control'])
        elif compare_group=='Normal-CRC_S1':
            class_compare = np.array([ 'Colorectal Cancer Stage 1', 'Healthy Control'])
        elif compare_group=='Normal-CRC_S1':
            class_compare = np.array([ 'Colorectal Cancer Stage 1', 'Healthy Control'])
        elif compare_group=='Normal-CRC_S1':
            class_compare = np.array([ 'Colorectal Cancer Stage 1', 'Healthy Control'])
    elif dataset =='lulab_hcc':
        if compare_group=='Normal-HCC':
            class_info = 'data_old/labels/sample_classes_hcc.txt'
            class_compare = np.array(['HCC', 'Normal']) 
        elif compare_group=='Normal-stage_A':
            class_compare = np.array(['stage_A', 'Normal']) 
    elif dataset =='exorbase':
        if compare_group=='Normal-HCC':
            class_compare = np.array(['HCC', 'Healthy']) 
        elif compare_group=='Normal-CRC':
            class_compare = np.array(['CRC', 'Healthy']) 
        elif compare_group=='Normal-PAAD':
            class_compare = np.array(['PAAD', 'Healthy']) 
    sample_class = pd.read_table(class_info,sep='\t',index_col=0)
    processed_matrix = pd.read_table('output/'+dataset+'/matrix_processing/'+preprocess_method+'.txt',sep='\t',index_col=0)
    feature_matrix[compare_group],feature_indicator_matrix[compare_group],\
    original_index[compare_group],foldchange[compare_group],rpkmtable[compare_group],cpmtable[compare_group] = compare_features(output_feature_selection_path+'{}/{}'.format(preprocess_method,compare_group),sample_class,class_compare,compare_group)
    features =  feature_matrix[compare_group].index
    cpm_table_use[compare_group],sample_class_use[compare_group],cpm_table_use_[compare_group] =  clustermap(processed_matrix,original_index[compare_group],sample_class,class_compare,compare_group,plot=0)
    cpm_table_use_[compare_group].to_csv(save_path+'selected_feature_cpm_'+compare_group+'.txt',sep='\t')

HBox(children=(IntProgress(value=0, max=3), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

random_forest.10.robust.any


,10,fold change
hsa-miR-1343-3p|miRNA|hsa-miR-1343-3p|hsa-miR-1343-3p,0.13,-0.31
30984|tRNA|peak_320|30984,0.09,0.34
C6orf226|mRNA|peak_805|ENST00000408925.3,0.12,-1.1
TAF1A|mRNA|peak_983|ENST00000465263.1,0.055,-1.6
chr1_172188440_172188480_-|genomic|peak_2041|chr1,0.14,-0.2
chr11_8044980_8045020_+|genomic|peak_2301|chr11,0.032,-1.6
chr16_85842820_85842860_-|genomic|peak_3067|chr16,0.14,-1.9
chr20_43244220_43244260_+|genomic|peak_3779|chr20,0.11,-1.7
chr3_98930080_98930120_+|genomic|peak_4065|chr3,0.052,-1.3
chrX_68305220_68305260_+|genomic|peak_5360|chrX,0.13,-2.2


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

random_forest.10.robust.any


,10,fold change
hsa-miR-92b-3p|miRNA|hsa-miR-92b-3p|hsa-miR-92b-3p,0.034,-0.18
22263|tRNA|peak_130|22263,0.077,0.13
24305|tRNA|peak_155|24305,0.044,0.15
ZNF831|mRNA|peak_670|ENST00000371030.3,0.092,-1.4
AC113554.2|mRNA|peak_1335|ENST00000577647.2,0.1,1
chr12_124921740_124921800_-|genomic|peak_2602|chr12,0.1,0.28
chr17_66755840_66755880_+|genomic|peak_3193|chr17,0.3,-0.93
chr17_72113840_72113880_-|genomic|peak_3198|chr17,0.067,-0.16
chr4_83237860_83237900_+|genomic|peak_4251|chr4,0.078,-1.4
chr5_37996200_37996240_-|genomic|peak_4408|chr5,0.1,0.41


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

random_forest.10.robust.any


,10,fold change
hsa-miR-130a-3p|miRNA|hsa-miR-130a-3p|hsa-miR-130a-3p,0.096,0.1
hsa-miR-410-3p|miRNA|hsa-miR-410-3p|hsa-miR-410-3p,0.027,0.088
35065|tRNA|peak_375|35065,0.054,-0.16
SNORD26|snoRNA|peak_721|ENST00000384147.1,0.14,-1.2
AC006548.3|lncRNA|peak_1541|ENST00000623130.1,0.048,-1.2
AC006548.3|lncRNA|peak_1543|ENST00000623130.1,0.073,-0.73
AC006548.3|lncRNA|peak_1545|ENST00000623130.1,0.16,-0.77
HELLPAR|lncRNA|peak_1612|ENST00000626826.1,0.24,-0.83
chr16_985540_985580_-|genomic|peak_2953|chr16,0.089,0.099
chr6_123272560_123272600_+|genomic|peak_4721|chr6,0.077,-0.064


In [123]:
filtered_mx = pd.read_table('output/'+dataset+'/matrix_processing/filter.'+exp_mx_name+'.txt')
filtered_mx.index =  np.array([filtered_mx.index[i].split('|')[2]+'|'+filtered_mx.index[i].split('|')[1]+'|'+filtered_mx.index[i].split('|')[3]+'|'+filtered_mx.index[i].split('|')[4] for i in range(filtered_mx.index.shape[0])])


In [135]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
viridisBig = cm.get_cmap('BuGn', 512)
newcmp = ListedColormap(viridisBig(np.linspace(0.4, 1, 256)))

def weight_bar_plt(weight=None,recurrence=None,rpkm=None,de=None,area_=(6.0,8.0),nameshort=True,savefig=True):
    weight.rename(columns={ weight.columns[0]: "weight" }, inplace=True)
    recurrence.rename(columns={ recurrence.columns[0]: "recurrence" }, inplace=True)
    de['name_cut'] = [featurename.split('|')[2] + '|' +\
                      featurename.split('|')[1] + '|' +\
                      featurename.split('|')[3] + '|' +\
                      featurename.split('|')[4]\
                      for featurename in de.index]
    de = de.set_index('name_cut')
    rpkm['name_cut'] = [featurename.split('|')[2] + '|' +\
                        featurename.split('|')[1] + '|' +\
                        featurename.split('|')[3] + '|' +\
                        featurename.split('|')[4]\
                        for featurename in rpkm.index]
    rpkm = rpkm.set_index('name_cut')
    weight = weight.sort_values('weight',ascending=False)
    index = weight.index
    plot_mx = pd.DataFrame({'weight':weight['weight'],
                  'rpkm':rpkm.loc[index].mean(axis=1),
                  'recurrence':recurrence.loc[index]['recurrence'],
                  'fc':de.loc[index]['log2FoldChange'],
                  'qvalue':de.loc[index]['padj']})
    plot_mx['-log10(qvalue)'] = -np.log10(plot_mx['qvalue']+0.01)
    plot_mx['log2RPKM'] = log_transfrom(plot_mx['rpkm'])

    if nameshort is True:
        plot_mx['nameshort'] = [i.split('|')[0] for i in index]
        plot_mx = plot_mx.set_index('nameshort')
        plot_mx.index.name=''
    fig, (ax_,ax, ax1) = plt.subplots(1,3, figsize=(7,3),gridspec_kw = {'width_ratios':[(1.5-nameshort)*2,2,2]})
    ax_.axis('off')
    #im = ax.scatter(plot_mx['weight'],plot_mx.index,s=((plot_mx['log2RPKM']/area_[0]-0.5)*area_[1])**2,c=plot_mx['recurrence'],cmap='BuGn')
    im = ax.scatter(plot_mx['weight'],plot_mx.index,s=((plot_mx['log2RPKM']/area_[0]-0.5)*area_[1])**2,c=plot_mx['recurrence'],cmap=newcmp)
    cbar =fig.colorbar(im, ax=ax)
    cbar.outline.set_visible(False)
    interval = np.max(plot_mx.log2RPKM) - np.min(plot_mx.log2RPKM)
    ratiointer = interval/4
    pws = set(np.round(np.arange(np.min(plot_mx.log2RPKM),np.max(plot_mx.log2RPKM),ratiointer),0).astype(int))
    for pw in pws:
        ax.scatter([], [], s=((pw/area_[0]-0.5)*area_[1])**2, c="k",label=str(pw))
    ax = std_plot(ax,'Feature Weight','Feature Name',None,'log(RPKM)',
                      borderpad=1,labelspacing=1.7,handletextpad=1,cbar=cbar,cbarlabel='Feature Recurrency',xlim=[0,np.max(plot_mx['weight'])+0.1])
                      #,ylim=[-0.5,nums_retain-0.5])
    #print(np.max(plot_mx['weight']))

    plot = ax1.scatter(plot_mx.index, plot_mx['fc'], c=plot_mx['-log10(qvalue)'], cmap='Reds')
    ax1.clear()
    sns.barplot(ax=ax1,data=plot_mx[::-1],y=plot_mx[::-1].index,x='fc',palette='Reds',hue='-log10(qvalue)',dodge=False)
    ax1.get_legend().remove()
    cbar = fig.colorbar(plot,ax=ax1)
    cbar.outline.set_visible(False)
    ax1 = std_plot(ax1,'Fold Change','','weight bar plot '+compare_group,moveyaxis=True,cbar=cbar,cbarlabel='-log10(q value)')
    ax1.set_yticks([])
    #fig.tight_layout()
    with embed_pdf_data() as data:
        fig.savefig(data, format='pdf', metadata={'Title': 'Weight Bar Plot '+compare_group})
    if savefig:
        fig.savefig(savepath+compare_group+'_weight_bar'+saveformat)
    return plot_mx.index
selected_features = {}
count = 0
scalefactor=np.array([0.6,0.8,0.5,0.5,0.5,0.5,0.5])*300
for compare_group, preprocess_method in tqdm(tmpdict.items()): 
    preprocess_method = preprocess_method[np.where(np.array([preprocess_method[i].split('.')[-1] for i in range(preprocess_method.shape[0])])==exp_mx_name)][0]
    
    if sequencing_type =='long':
        tmptable = rpkmtable[compare_group]
    else:
        tmptable = cpmtable[compare_group]
    detable = pd.read_table('output/'+dataset+'/differential_expression/'+exp_mx_name+'/'+compare_group+'/deseq2.txt'
                       ,index_col=0) 
    #display(detable)
    #print ('output/'+dataset+'/cross_validation/'+preprocess_method+'/'+compare_group+'/'+classifier_use+'.10.robust.'+value_change+'/cross_validation.h5')
    with h5py.File('output/'+dataset+'/cross_validation/'+preprocess_method+'/'+compare_group+'/'+classifier_use+'.10.robust.'+value_change+'/cross_validation.h5') as f:
        feature_selection_matrix = f['feature_selection'][:]
    tmprecurrencedf = pd.DataFrame(feature_selection_matrix.mean(axis=0))
    tmprecurrencedf.index = filtered_mx.index                                 
    detable=detable.fillna(1)
    selected_features[compare_group] =weight_bar_plt(feature_matrix[compare_group],tmprecurrencedf,tmptable,detable,(6.0,8.0),False)
    #selected_features[compare_group] = feature_weight_bar(feature_matrix[compare_group],detable,tmprecurrencedf,
      #            10,original_index=original_index[compare_group],rpkmtable=tmptable,
      #                           compare_group=compare_group,scalefactor=scalefactor[count],area_=(8.0,9.0),namelength=3)
    count+=1

HBox(children=(IntProgress(value=0, max=3), HTML(value='')))

In [125]:
def find_metrics_best_for_shuffle(fpr,tpr):
    '''
    used for shuffle roc plot
    '''
    a = 1 - fpr 
    b = tpr
    Sensitivity = b
    Specificity = a
    arith_mean = (Sensitivity+Specificity)*0.5
    geo_mean = (Sensitivity*Specificity)**0.5
    harmo_mean = 2/(1/Sensitivity+1/Specificity)
    eucilid_mean = ((1-Sensitivity)**2+(1-Specificity)**2)**0.5
    auc = sklearn.metrics.auc(a,b)
    thres = 1
    uni,counts= np.unique(np.concatenate((np.argsort(-arith_mean)[:thres],np.argsort(-geo_mean)[:thres],np.argsort(-harmo_mean)[:thres],
                np.argsort(-eucilid_mean)[:thres])),return_counts=True)
    #print (np.max(counts))
    if np.where(counts ==np.max(counts))[0].shape[0] >1:
        ind =1
    else:
        ind =0
    return auc,Sensitivity[uni[np.where(counts ==np.max(counts))][ind]],Specificity[uni[np.where(counts ==np.max(counts))][ind]]
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def get_probs_split(filename,interpolatednum=10000,confidence=0.95):
    '''
    get probs from split
    use interpolation to calculate CI
    '''
    xvals = np.linspace(0, 1, interpolatednum)
    with h5py.File(filename,'r') as f:
        predictions =  f['predictions'][:]
        train_index =  f['train_index'][:]
        label = f['labels'][:]
    probs,labels,interpolatedvalue = {},{},{}
    for i in range(predictions.shape[0]):
        probs[i] = predictions[i][~train_index[i]]
        labels[i] = label[~train_index[i]]
        interpolatedvalue[i] = np.interp(xvals,plot_roc( probs[i],labels[i])[1],plot_roc( probs[i],labels[i])[2] )
    interarray = np.array([interpolatedvalue[i]  for i in range(predictions.shape[0])])
    mean,minimum,maximum = np.ndarray([interpolatednum]),np.ndarray([interpolatednum]),np.ndarray([interpolatednum])
    for i in range(interpolatednum):
        mean[i],minimum[i],maximum[i] = mean_confidence_interval(interarray[:,i],confidence)
    return np.concatenate((np.zeros(1),mean,np.ones(1))),np.concatenate((np.zeros(1),minimum,np.ones(1))),\
np.concatenate((np.zeros(1),maximum,np.ones(1))),interpolatednum+2
def plot_roc(prob,label):
    fpr, tpr, _ = roc_curve(label, prob)
    roc_auc = sklearn.metrics.auc(fpr, tpr)
    return roc_auc,fpr, tpr
def find_metrics_best(label,expressionlevel):
    posinum = np.sum(label)
    neganum = label.shape[0] - np.sum(label)
    a,b,c= roc_curve(label,expressionlevel) #fpr tpr threshold
    fp = a*neganum
    tp = b*posinum
    tn = (1-a)*neganum
    fn = (1-b)*posinum
    #Sensitivity = tp/(tp+fn) 
    #Specificity = tn/(tn+fp) 
    Sensitivity = b
    Specificity = 1 - a
    PPV = tp/(tp+fp) 
    NPV = tn/(tn+fn) 
    arith_mean = (Sensitivity+Specificity)*0.5
    geo_mean = (Sensitivity*Specificity)**0.5
    harmo_mean = 2/(1/Sensitivity+1/Specificity)
    eucilid_mean = ((1-Sensitivity)**2+(1-Specificity)**2)**0.5
    mcc_mean = (tp*tn-fp*fn)/(((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))**0.5)
    F1_score = 2*tp/(2*tp+fn+fp)
    auc = sklearn.metrics.auc(a,b)
    acc = (tp+tn)/(tp+tn+fn+fp)
    distoleftup = (a**2+(1-b)**2)**0.5# to the left up
    #print ("AUC:" +str(auc)) 
    thres = int(label.shape[0]/10.) if int(label.shape[0]/10.)>0 else 1
    uni,counts= np.unique(np.concatenate((np.argsort(-arith_mean)[:thres],np.argsort(-geo_mean)[:thres],np.argsort(-harmo_mean)[:thres],
                np.argsort(-eucilid_mean)[:thres],np.argsort(-mcc_mean)[:thres],np.argsort(-F1_score)[:thres])),return_counts=True)
    #print (np.max(counts))
    if np.where(counts ==np.max(counts))[0].shape[0] >1:
        ind =1
    else:
        ind =0
    return auc,Sensitivity[uni[np.where(counts ==np.max(counts))][ind]],Specificity[uni[np.where(counts ==np.max(counts))][ind]]
def get_auc_stats(root_path,sample_path):
    '''
    sample_path:hccpath,stagepath
    '''
    auc_metrics_jack = {}
    auc_metrics_split = {}
    for i in os.listdir(root_path+sample_path):
        #print (i)
        try:
            print (i, ' file found')
            if i.split('.')[-1] == 'shuffle_split':
                auc_metrics_split[i] = h5py.File(root_path+sample_path+i+'/evaluation.shuffle_split.h5')
            else:
                with h5py.File(root_path+sample_path+i+'/evaluation.leave_one_out.h5','r') as f:
                    predictions =  f['predictions'][:]
                    train_index =  f['train_index'][:]
                    auc_metrics_jack[i]  = predictions[~train_index]
        except:
            print (i, ' file not found')
            print ('files in the directory: ',os.listdir(root_path+sample_path+i))
    return auc_metrics_jack,auc_metrics_split
def get_speci_sensi(filename,spe_criteria=0.99):
    mean,minimum,maximum,interpolatednum = get_probs_split(filename)
    total_nums = mean.shape[0]
    index = round(total_nums *(1-spe_criteria))
    return mean[index],minimum[index],maximum[index]

In [126]:
def plot_shuffle(ax,filename,filename_,show_metrics=False,savefigure=False,title=None,
                 spe_criter=0.99,savepath=savepath,savefig=True,
                 compare_group=None,markers_file=None,marker_roc_ind = np.array([0]),markername=None,
                show_up_value=True):
    '''
    samplepath: hccpath
    filename: 'logistic_regression.100.robust.stratified_shuffle_split/evaluation.stratified_shuffle_split.h5'
    '''  
    if markers_file is not None:
        markers_auc = {}
        for i in range(markers_file.shape[0]):
            mean,_,_,interpolatednum = get_probs_split(markers_file[i])
            xvals = np.linspace(0, 1, interpolatednum)  
            auc,sen,spe = find_metrics_best_for_shuffle(xvals,mean) #tpr,1-fpr
            if np.isin(i,marker_roc_ind):
               # print (markername[marker_roc_ind[i]],marker_roc_ind[i])
                ax.plot(xvals,mean,linewidth=1.5,color='gray',label=markername[i]+' AUC: '+str(np.round(auc,3)))
            markers_auc[i],_,_ = find_metrics_best_for_shuffle(xvals,mean)
    mean,minimum,maximum,interpolatednum = get_probs_split(filename)
    xvals = np.linspace(0, 1, interpolatednum)
    #ax.fill_between(xvals,minimum,maximum,where=(minimum<=maximum),color='b',alpha=0.1)   
    auc,sen,spe = find_metrics_best_for_shuffle(xvals,mean) #tpr,1-fpr
    candidatevalue = auc
    numind = np.ceil((1-spe)/(1./(interpolatednum-2))).astype('int')
    ax.plot(xvals,mean,linewidth=2,color=Category20c[20][1],label='candidate AUC: '+str(np.round(auc,3)))

    if show_up_value:
        mean,minimum,maximum,interpolatednum = get_probs_split(filename_)
        xvals = np.linspace(0, 1, interpolatednum)
        #ax.fill_between(xvals,minimum,maximum,where=(minimum<=maximum),color='b',alpha=0.1)   
        auc,sen,spe = find_metrics_best_for_shuffle(xvals,mean) #tpr,1-fpr
        candidatevalue = auc
        numind = np.ceil((1-spe)/(1./(interpolatednum-2))).astype('int')
        ax.plot(xvals,mean,linewidth=2,color=Category20c[20][10],label='candidate direction up AUC: '+str(np.round(auc,3)))

        
    h,l = ax.get_legend_handles_labels()
    legend = ax.legend(h,l)
    sen_criter = get_speci_sensi(filename,spe_criteria=spe_criter)[0]
    if show_metrics:
        if sen+0.2<=0.95:
            ax.annotate('Specificity '+str('%0.4s'%(spe)), xy=( 1-spe,sen), xytext=(1-spe, sen+0.2),size=6.5,
            arrowprops=dict(facecolor='black', shrink=0.05,width=1.5,headwidth=2.5),
            )
        else:
            ax.annotate('Specificity '+str('%0.4s'%(spe)), xy=( 1-spe,sen), xytext=(1-spe+0.05, sen-0.2),size=6.5,
            arrowprops=dict(facecolor='black', shrink=0.05,width=1.5,headwidth=2.5),
            )
        ax.annotate('Sensitivity '+str('%0.4s'%sen), xy=( 1-spe,sen), xytext=(1-spe+0.1, sen-0.1),size=6.5,
            arrowprops=dict(facecolor='black', shrink=0.1,width=1.5,headwidth=2.5),
            )
    ax.annotate('Sensitivity '+str('%0.4s'%sen_criter), xy=( 1-spe_criter,sen_criter), xytext=(1-spe_criter+0.1, sen_criter),size=6.5,
        arrowprops=dict(facecolor='black', shrink=0.8,width=1.5,headwidth=2.5),
        )
    ax = std_plot(ax,'False Positive Rate','True Positive Rate',title,'Marker',legendsort=False,borderpad=0.4,handletextpad=0.4)
    ax.plot([0,1],[0,1], linewidth=2,alpha=0.6,color='gray',linestyle='--')
    #ax.set_yticklabels(np.arange(0,12,2)/10,fontticklabel)
    #ax.set_xticklabels(np.arange(0,12,2)/10,fontticklabel)
    if savefig:
        fig.savefig(savepath+compare_group+'_ROC'+saveformat, bbox_inches='tight')
    #fig.tight_layout()
    with embed_pdf_data() as data:
        fig.savefig(data, format='pdf', metadata={'Title': '123'})
    #return mean,minimum,maximum,interpolatednum
    
    return markers_auc,candidatevalue

In [127]:
def return_marker_file(dataset,compare_group,num,classifier_use,preprocess_method,de=True):
    marker_file = np.array(['output/'+dataset+'/evaluate_features/control_markers_'+str(j)+\
        '/'+preprocess_method+'/'+compare_group+'/'+classifier_use+'/cross_validation.h5' for j in range(num) ])
    if de==True:
        marker_file = np.concatenate((marker_file,np.array(['output/'+dataset+'/evaluate_features/result/'+dataset+'.de_selected_mx/'+compare_group+
                                                            '/random_forest/evaluation.stratified_shuffle_split.h5'])))
    return marker_file

In [129]:
control_markers = {}
control_markers['Normal-CRC'] = {}
#control_markers['Normal-CRC'][0] = np.array(['hsa-miR-92a-3p','hsa-miR-92b-3p',
 #                                            'hsa-miR-92b-5p','hsa-miR-21-3p',
  #                                          'hsa-miR-21-5p','hsa-miR-139-3p',
    #                                        'hsa-miR-139-5p','hsa-miR-431-3p','hsa-miR-431-5sp'])
control_markers['Normal-CRC'][1] =np.array(['hsa-miR-20a-5p'])
control_markers['Normal-CRC'][2] =np.array(['hsa-miR-26a-5p'])
control_markers['Normal-CRC'][3] =np.array(['hsa-miR-124-5p'])
control_markers['Normal-CRC'][4] =np.array(['hsa-miR-183-5p'])
control_markers['Normal-CRC'][5] =np.array(['hsa-miR-221-3p','hsa-miR-221-5p'])
control_markers['Normal-CRC'][6] =np.array(['hsa-miR-200c-3p'])
control_markers['Normal-CRC'][7] =np.array(['hsa-miR-25-3p'])
control_markers['Normal-CRC'][8] =np.array(['hsa-miR-331-3p'])
control_markers['Normal-CRC'][9] =np.array(['hsa-miR-103a-3p'])
control_markers['Normal-CRC'][10] =np.array(['hsa-miR-127-3p'])
control_markers['Normal-CRC'][11] =np.array(['hsa-miR-151a-5p'])
control_markers['Normal-CRC'][12] =np.array(['hsa-miR-17-5p'])
control_markers['Normal-CRC'][13] =np.array(['hsa-miR-181a-5p'])
control_markers['Normal-CRC'][14] =np.array(['hsa-miR-19b-3p'])
control_markers['Normal-CRC'][15] =np.array(['hsa-miR-15b-5p','hsa-miR-15b-3p'])
control_markers['Normal-CRC'][16] =np.array(['hsa-miR-29a-3p'])
control_markers['Normal-CRC'][17] =np.array(['hsa-miR-335-3p','hsa-miR-335-5p'])
#control_markers['Normal-CRC'][18] =np.array(['hsa-miR-92b-3p','hsa-miR-92b-5p','hsa-miR-92a-3p','hsa-miR-92a-5p'])
control_markers['Normal-CRC_S1'] = control_markers['Normal-CRC']

control_markers['Normal-PAAD'] = {}
control_markers['Normal-PAAD'][0] = np.array(['hsa-miR-106b-3p','hsa-miR-106b-5p'])
control_markers['Normal-PAAD'][1] = np.array(['hsa-miR-10b-3p','hsa-miR-10b-5p'])
control_markers['Normal-PAAD'][2] = np.array(['hsa-miR-21-5p','hsa-miR-21-3p'])
control_markers['Normal-PAAD'][3] = np.array(['hsa-miR-30c-5p'])
control_markers['Normal-PAAD'][4] = np.array(['hsa-miR-20a-5p'])
control_markers['Normal-PAAD'][5] = np.array(['hsa-miR-181a-3p','hsa-miR-181a-5p'])
control_markers['Normal-PAAD'][6] = np.array(['hsa-miR-181a-3p'])
control_markers['Normal-PAAD'][7] = np.array(['hsa-miR-22-5p','hsa-miR-22-3p'])
control_markers['Normal-PAAD'][8] = np.array(['hsa-miR-885-5p','hsa-miR-885-3p'])
control_markers['Normal-PAAD'][9] = np.array(['hsa-miR-744-5p','hsa-miR-744-3p'])
control_markers['Normal-PAAD'][10] = np.array(['hsa-miR-10b-3p','hsa-miR-10b-5p'])
#control_markers['Normal-PAAD'][11] = np.array(['hsa-miR-30c-5p'])
control_markers['Normal-PAAD'][12] = np.array(['hsa-miR-106b-3p','hsa-miR-106b-5p'])
control_markers['Normal-PAAD'][13] = np.array(['hsa-miR-155-5p'])
control_markers['Normal-PAAD'][14] = np.array(['hsa-miR-212-3p','hsa-miR-212-5p'])
control_markers['Normal-PAAD'][15] = np.array(['hsa-miR-10b-3p','hsa-miR-10b-5p','hsa-miR-106b-3p','hsa-miR-106b-5p'])
control_markers['Normal-PAAD'][16] = np.array(['hsa-miR-221-5p','hsa-miR-221-3p'])
control_markers['Normal-PAAD'][17] = np.array(['hsa-miR-210-3p'])
#control_markers['Normal-PAAD'][18] = np.array(['hsa-miR-21-5p','hsa-miR-21-3p','hsa-miR-210-3p','hsa-miR-155-5p'])

control_markers['Normal-PRAD'] = {}
control_markers['Normal-PRAD'][0] = np.array(['hsa-miR-30c-5p'])
control_markers['Normal-PRAD'][1] = np.array(['hsa-miR-26b-3p','hsa-miR-26b-5p'])
control_markers['Normal-PRAD'][2] = np.array(['hsa-miR-451a'])
control_markers['Normal-PRAD'][3] = np.array(['hsa-miR-24-3p'])
control_markers['Normal-PRAD'][4] = np.array(['hsa-miR-874-3p'])
control_markers['Normal-PRAD'][5] = np.array(['hsa-miR-93-5p'])
control_markers['Normal-PRAD'][6] = np.array(['hsa-miR-106a-5p'])
control_markers['Normal-PRAD'][7] = np.array(['hsa-let-7e-3p','hsa-let-7e-5p','hsa-miR-30c-5p','hsa-miR-1285-3p'])
control_markers['Normal-PRAD'][8] = np.array(['hsa-miR-210-3p','hsa-miR-210-5p'])
control_markers['Normal-PRAD'][9] = np.array(['hsa-miR-221-3p','hsa-miR-221-5p'])
control_markers['Normal-PRAD'][10] = np.array(['hsa-miR-375-3p'])
control_markers['Normal-PRAD'][11] = np.array(['hsa-miR-107'])
control_markers['Normal-PRAD'][12] = np.array(['hsa-miR-130b-3p','hsa-miR-130b-5p'])
control_markers['Normal-PRAD'][13] = np.array(['hsa-miR-181a-2-3p'])
control_markers['Normal-PRAD'][14] = np.array(['hsa-miR-2110'])
control_markers['Normal-PRAD'][15] = np.array(['hsa-miR-301a-5p'])
control_markers['Normal-PRAD'][16] = np.array(['hsa-miR-326'])
control_markers['Normal-PRAD'][17] = np.array(['hsa-miR-331-3p'])
control_markers['Normal-PRAD'][18] = np.array(['hsa-miR-432-5p'])
control_markers['Normal-PRAD'][19] = np.array(['hsa-miR-484'])
control_markers['Normal-PRAD'][20] = np.array(['hsa-miR-574-3p'])
control_markers['Normal-PRAD'][21] = np.array(['hsa-miR-625-3p'])



In [131]:
marker_roc_ind = {}
marker_roc_ind['Normal-CRC'] = np.array([ 11,16 ])
marker_roc_ind['Normal-CRC_S1'] = np.array([ 10,11 ])
marker_roc_ind['Normal-PRAD'] = np.array([3,10])

In [136]:
tmpdict

{'Normal-CRC': array(['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg',
        'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna',
        'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains'],
       dtype=object),
 'Normal-CRC_S1': array(['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg',
        'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna',
        'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains'],
       dtype=object),
 'Normal-PRAD': array(['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg',
        'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna',
        'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains'],
       dtype=object)}

In [132]:
markers_auc = {}
candidatevalue={}

#for i in range(1):
 #   compare_group = 'Normal-CRC' 
 #   preprocess_method = tmpdict['Normal-CRC']
for compare_group, preprocess_method in tqdm(tmpdict.items()):
    preprocess_method = preprocess_method[np.where(np.array([preprocess_method[i].split('.')[-1] for i in range(preprocess_method.shape[0])])==exp_mx_name)][0]
    
    shuffle_split_roc_file = 'output/'+dataset+'/cross_validation/'+preprocess_method+'/'+compare_group+'/'+classifier_use+'.10.robust.'+value_change+'/cross_validation.h5'
    shuffle_split_roc_file_up = 'output/'+dataset+'/cross_validation/'+preprocess_method+'/'+compare_group+'/'+classifier_use+'.10.robust.up/cross_validation.h5'
    fig,ax=plt.subplots(1,figsize=(4,4))
    #markername  = np.array([control_markers[compare_group][i][0] for i in control_markers[compare_group].keys()])
    markername  = {}
    count = 0
    if compare_group[:10] =='Normal-CRC':
        control_markers[compare_group] = control_markers['Normal-CRC']
    for i in control_markers[compare_group].keys():
        tmpmarkers = np.unique(np.array([control_markers[compare_group][i][j][:-3] for j in range(control_markers[compare_group][i].shape[0])]))
        if tmpmarkers.shape[0] >1:
            print (tmpmarkers)
            markername[i] = 'Panel '+str(count+1)
            count+=1
        else:
            markername[i] = tmpmarkers[0]
    #print (markername)
    ## TO DO: panel
    if compare_group=='Normal-CRC':
        markers_auc[compare_group],candidatevalue[compare_group] =  plot_shuffle(ax,shuffle_split_roc_file,shuffle_split_roc_file_up,
                     spe_criter=0.99,compare_group=compare_group,
                    markers_file=return_marker_file(dataset,compare_group,len(control_markers[compare_group].keys()),classifier_use,preprocess_method,de=False)
                #markers_file=return_marker_file(dataset,compare_group,1,de=False)                        
                           ,title= 'ROC of '+compare_group,marker_roc_ind =marker_roc_ind[compare_group],markername=markername)
    else:
        #dataset,compare_group,num,classifier_use,preprocess_method,
        markers_auc[compare_group],candidatevalue[compare_group] = plot_shuffle(ax,shuffle_split_roc_file,shuffle_split_roc_file_up,
                     spe_criter=0.99,compare_group=compare_group,
                   markers_file=return_marker_file(dataset,compare_group,len(control_markers[compare_group].keys()),classifier_use,preprocess_method,de=False)
                #markers_file=return_marker_file(dataset,compare_group,0,de=False)  
                         ,title= 'ROC of '+compare_group,marker_roc_ind =marker_roc_ind[compare_group],markername=markername)
        

HBox(children=(IntProgress(value=0, max=3), HTML(value='')))

['hsa-let-7e' 'hsa-miR-1285' 'hsa-miR-30c']


In [133]:
### pd.set_option('display.max_colwidth', -1)
for compare_group, preprocess_method in tqdm(tmpdict.items()):
    print (compare_group, preprocess_method)
    if len(control_markers[compare_group].keys())>0:
        #print (len(control_markers[compare_group].keys()),compare_group)
        #markernames  = np.concatenate((np.array(['Candidate']),np.array([control_markers[compare_group][i][0] for i in range(len(control_markers[compare_group].keys()))])))
        markernames = np.ndarray([len(control_markers[compare_group].keys())+1]).astype('str')
        markernames[0] = 'Candidate'
        count = 0
        count_=1
        panel_marker = []
        for i in control_markers[compare_group].keys(): 
            tmpmarkers = np.unique(np.array([control_markers[compare_group][i][j][:-3] for j in range(control_markers[compare_group][i].shape[0])]))
            if tmpmarkers.shape[0] >1:

                markernames[count_] = 'Panel '+str(count)
                panel_marker.append(np.array2string(control_markers[compare_group][i],separator=',')[2:-2].replace("'", ""))
                count+=1
            else:
                markernames[count_] = control_markers[compare_group][i][0]
            count_+=1
        #print (markernames)
        if len(panel_marker) >0:
            panel_table = pd.DataFrame(panel_marker)
            panel_table.index = np.array(['panel '+str(i) for i in range(panel_table.shape[0])])
            panel_table.columns = ['markers']
            display(panel_table)
        selectedind = np.array([i for i in control_markers[compare_group].keys()])
        #print (selectedind)
        
        aucall = np.concatenate((np.array([candidatevalue[compare_group]]),np.array([markers_auc[compare_group][i] for i in markers_auc[compare_group].keys()])))
        auctable =pd.DataFrame(np.concatenate((markernames.reshape(-1,1),aucall.reshape(-1,1)),axis=1))
        auctable.columns = ['Marker Name','AUROC']
        auctable.AUROC = pd.to_numeric(auctable.AUROC )
        auctable = auctable.sort_values('AUROC',ascending=0)
        #display(auctable)
        fig, ax = plt.subplots(1,figsize=(7,4))
        #auctable.plot(kind='bar')
        
        clrs = ['grey' if (x < max(np.array(auctable.AUROC))) else 'blue' for x in np.array(auctable.AUROC)]
        sns.barplot(data=auctable,x='Marker Name',y='AUROC',palette=clrs)
        fig.tight_layout()
        ax.set_xticklabels(auctable.iloc[:,0],rotation=90,fontsize=20,weight='bold')
        ax = std_plot(ax,'Marker Name','AUROC','Bar plot of markers AUC '+compare_group)
        #ax.set_ylim(0,1)
        #if savefig:
        fig.savefig(savepath+compare_group+'_marker_barplot'+saveformat, bbox_inches='tight')
        
        with embed_pdf_data() as data:
            fig.savefig(data, format='pdf', metadata={'Title': '123'})
        fig.tight_layout()

HBox(children=(IntProgress(value=0, max=3), HTML(value='')))

Normal-CRC ['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg'
 'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna'
 'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains']


Normal-CRC_S1 ['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg'
 'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna'
 'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains']


Normal-PRAD ['filter.null.Norm_CPM.Batch_Combat_1.mirna_and_long_bg'
 'filter.null.Norm_CPM.Batch_Combat_1.transcript_mirna'
 'filter.null.Norm_CPM_top.Batch_Combat_1.mirna_and_domains']


,markers
panel 0,"hsa-let-7e-3p,hsa-let-7e-5p,hsa-miR-30c-5p,hsa..."
